In [ ]:

# 1. Install Dependencies - NOTE: The 'minimal' image/kernel we are using actually has all of these already.
# It is ok to attempt these installs and it is better to attempt them than assume the image has them all.
# NOTE: Include the % at the start to ensure installation into the correct kernel/environment. (in .ipynb files)
%pip install requests --quiet
%pip install pandas --quiet
%pip install geopandas --quiet
%pip install mapboxgl --quiet
%pip install shapely --quiet


In [ ]:
# 2. Setup / Configuration
import os
import requests


# TODO: Currently using default value as I don't have the needed ENV hooked up here yet. OK for BASE_URL.
BASE_URL = os.environ.get('BASE_URL', 'http://linkspeed-fastapi:8000')
print(f"BASE_URL: {BASE_URL}")

# TODO: Currently using default value as I don't have the needed ENV hooked up here yet. OK for BASE_URL.
#  For a secure API token, this is very undesireable for security as well as for repository integrity.
#  Ideally I will be able to get ENV from the Docker Compose ENV which will get it from a .env file etc.
MAPBOX_TOKEN = os.environ.get(
        'MAPBOX_TOKEN',
        'pk.eyJ1Ijoic21hcnRtZXRhbCIsImEiOiJjbWJnbnRhbHIwMHN5MmpvaDN5M2E3dnptIn0.OOMN3laXveAb-ep2U2WqsQ'
    )
print(f"MAPBOX_TOKEN: {MAPBOX_TOKEN}")


In [ ]:
# 3. Request aggregated data
# Aggregates URL: BASE_URL/aggregates/

params = {
    'day': 'Wednesday',
    'period': 'pm_peak',
}

# ONE SAMPLE - link_id = 16981048
with requests.Session() as session:
    session.headers.update({'User-Agent': 'LinkSpeed Assignment Jupyter Notebook - traffic.ipynb'})
    response = session.request(method='GET', url=f"{BASE_URL}/aggregates/16981048", params=params)
    print(response.status_code)
    print(response.text)


# ALL
geojson_data = []
with requests.Session() as session:
    session.headers.update({'User-Agent': 'LinkSpeed Assignment  Jupyter Notebook - traffic.ipynb'})
    response = session.request(method='GET', url=f"{BASE_URL}/aggregates", params=params)
    print(response.status_code)
    # print(response.text)  # Usually more than you want to print or scroll through.
    geojson_data = response.json()



In [ ]:
# 4. Visualize in Mapbox
from mapboxgl.viz import ChoroplethViz
from mapboxgl.utils import create_color_stops

# Assuming geojson_data is a list of Feature dicts
features = [{
    "type": "Feature",
    "geometry": f["geometry"],
    "properties": {
        "average_speed": f["properties"]["average_speed"],
        "road_name": f["properties"]["road_name"]
    }
} for f in geojson_data]

viz = ChoroplethViz(
    {
        "type": "FeatureCollection",
        "features": features
    },
    access_token=MAPBOX_TOKEN,
    color_property="average_speed",
    color_stops=create_color_stops([10, 20, 30, 40, 50], colors="Reds"),
    center=(-81.6557, 30.3322),
    zoom=11,
    line_width=1.5,
    opacity=0.8,
    #legend_title="Average Speed (mph)"
)

viz.show()
